In [275]:
import os

os.chdir(fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3')

os.getcwd() 

'\\\\?\\D:\\Projects\\Calm_ISO\\5h_Injection\\ISO_Total'

In [276]:
# creates a list of paths with labeled images

def get_labeled_puncta_path(mNRA_Red, SubSET=None):

    WORD1 = str(mNRA_Red).lower()
    WORD2 = str(SubSET).lower()

    input_dir = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3'

    labeled_path_list = []

    # Loop through all directories and subdirectories inside the input directory
    for root, dirs, files in os.walk(input_dir):
        # Loop through all files in the current directory
        for file in files:
            # Check if the file has an _info.txt extension and starts with the folder name
            if "_info.txt" in file and file.startswith(os.path.basename(root)):
                # Read the file content and convert it to lowercase
                with open(os.path.join(root, file), "r") as f:
                    file_content = f.read().lower()

                # Check if WORD1 and WORD2 are in the file content
                if WORD1 in file_content and WORD2 in file_content:
                # if WORD1 in file_content:
                    # Append the folder path to the list
                    labeled_path_list.append(root)

    # Return the list of paths+
    return labeled_path_list



In [277]:
labeled_puncta_path_list = get_labeled_puncta_path("Ryr2", "WT")

len(labeled_puncta_path_list)
# labeled_puncta_path_list

39

In [279]:
from concurrent.futures import ThreadPoolExecutor
from skimage import io, util, morphology
import os
import re
import numpy as np
from skimage.measure import label, regionprops
import tifffile
import pandas as pd
import time
from sklearn.neighbors import NearestNeighbors
import math
import json
import pickle as pkl

input_dir = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3'
os.chdir(input_dir)

def Puncta_Processing(path_list, mRNA1, SubSet=None):
       
    df = pd.DataFrame()
    df_red_to_nuclei_distances_raw = pd.DataFrame()
    df_red_to_nuclei_distances_normalized = pd.DataFrame()
    
    df_random_red_to_nuclei_distances_raw = pd.DataFrame()
    df_random_red_to_nuclei_distances_normalized = pd.DataFrame()
   
    df_membrane_to_nuclei_distances_raw = pd.DataFrame()     
    df_membrane_to_nuclei_distances_normalized = pd.DataFrame()

    WORD1 = str(mRNA1) 
    WORD2 = str(SubSet)
    
    for path in path_list:
        min_distances = []  # list to store min distances
        min_distances_normalized = []  # list to store normalized min distances
        red_to_nuclei_distances = {}        

        start_time = time.time()  # start the timer
        
        mRNA1_Name_SubSET_mask_path = os.path.join(path, 'Segmented', f'Labeled_{WORD1}.tif')
        Body_SubSET_mask_path = os.path.join(path, 'Segmented', 'Masked_Body_image.tif')
        Nuclei_SubSET_mask_path = os.path.join(path, 'Segmented', 'Masked_Nuclei_image.tif')
        Cytosol_SubSET_mask_path = os.path.join(path, 'Segmented', 'Masked_Cytosol_image.tif')
        mRNA1_Name_SubSET_mask_path_bool = os.path.join(path, 'Segmented', f'Masked_{WORD1}_filtered_image.tif')

        with open(os.path.join(path, os.path.basename(path) + '_info.txt'), 'r', encoding='utf-8', errors='ignore') as file:
            text = file.read()

        pattern = r"XYZCalibration\(um\/vox\):(\d+\.\d+),(\d+\.\d+),(\d+\.\d+):"
        match = re.search(pattern, text)

        if match:
            x_dim = float(match.group(1))
            y_dim = float(match.group(2))
            z_dim = float(match.group(3))
        else:
            print("No match found.")  

        voxel_dims = (z_dim, y_dim, x_dim)
        
        # Load the binary images
        red_img = tifffile.imread(mRNA1_Name_SubSET_mask_path) # Centroids with Zero coordinates
        nuclei_img = tifffile.imread(Nuclei_SubSET_mask_path)
        cytosol = util.img_as_bool(tifffile.imread(Cytosol_SubSET_mask_path))
        mRNA1_bool = util.img_as_bool(tifffile.imread(mRNA1_Name_SubSET_mask_path_bool))
        
        Body_bool = util.img_as_bool(tifffile.imread(Body_SubSET_mask_path))
        dilated_Body_bool = morphology.binary_dilation(Body_bool)
        edges_Body_bool = np.logical_xor(Body_bool, dilated_Body_bool)
#         tifffile.imwrite('edges_Body_bool.tif', edges_Body_bool)
        Nuclei_bool = util.img_as_bool(nuclei_img)
        
        # Find the centroids of the red puncta
        red_labels = label(red_img)
        red_props = regionprops(red_labels)
        
        # Extract the centroids and store them in a list
        red_centroids = []
        for prop in red_props:
            z, y, x = prop.centroid
            centroid = (z * z_dim, y * y_dim, x * x_dim)
            red_centroids.append(centroid)
        
        # Get the coordinates of all True values in Nuclei and Edges of Cell Body
        coords_Nuclei = np.argwhere(Nuclei_bool == True)
        coords_edges_Body = np.argwhere(edges_Body_bool == True)  
        
        # Scale the coordinates to physical units using the voxel dimensions
        coords_Nuclei_physical = coords_Nuclei * voxel_dims
        coords_edges_Body_physical = coords_edges_Body * voxel_dims
        
        # Initialize the NearestNeighbors object with n_neighbors=1 (find the nearest neighbor) 
        nn1 = NearestNeighbors(n_neighbors=1, algorithm='kd_tree', metric='euclidean')
        nn1.fit(coords_Nuclei_physical) #mRNA_Name to Nuclei fit
        distances, indices = nn1.kneighbors(coords_edges_Body_physical)
        temp_df3 = pd.DataFrame({'z': coords_edges_Body_physical[:, 0], 'y': coords_edges_Body_physical[:, 1], 'x': coords_edges_Body_physical[:, 2], 'distance': distances.flatten()}) # Create a pandas dataframe to store the results
        max_distance_from_nuclei = temp_df3['distance'].max() 
        
        min_distances_membrane_normalized = temp_df3['distance'] / max_distance_from_nuclei
        df_min_distances_membrane_normalized = pd.DataFrame(min_distances_membrane_normalized)
        df_min_distances_membrane_normalized.columns = [os.path.basename(path)]
#         df_min_distances_membrane_normalized = pd.DataFrame(min_distances_membrane_normalized, columns=[os.path.basename(path)])
        df_membrane_to_nuclei_distances_normalized = pd.concat([df_membrane_to_nuclei_distances_normalized, df_min_distances_membrane_normalized], axis=1)
        median_distance_membrane_normalized = np.quantile(min_distances_membrane_normalized, 0.5)

        min_distances_membrane_raw = temp_df3['distance']
        df_min_distances_membrane_raw = pd.DataFrame(min_distances_membrane_raw)
        df_min_distances_membrane_raw.columns = [os.path.basename(path)]
        df_membrane_to_nuclei_distances_raw = pd.concat([df_membrane_to_nuclei_distances_raw, df_min_distances_membrane_raw], axis=1)
        median_distance_membrane_raw = np.quantile(min_distances_membrane_raw, 0.5)

        # Calculate distances from red centroids to the nearest nuclei pixel
        if red_centroids and len(coords_Nuclei_physical) > 0:
            red_centroids_array = np.array(red_centroids)
            nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(coords_Nuclei_physical)
            distancesNN, indices = nbrs.kneighbors(red_centroids_array)

            min_distances = [] # list to store min distances
            min_distances_normalized = [] # list to store min distances            
            
            for i, red_coord in enumerate(red_centroids):
                nuclei_coord = coords_Nuclei_physical[indices[i][0]]
                min_distance = distancesNN[i][0]
                red_to_nuclei_distances[red_coord] = {'nuclei_coordinates': tuple(nuclei_coord), 'distance': min_distance}

                min_distances.append(min_distance) # append min distance to the list
                min_distances_normalized.append(min_distance / max_distance_from_nuclei) # append min distance to the list

            # Calculate the median for min_distances and min_distances_normalized for the current path
            median_distance_raw = np.quantile(min_distances, 0.5)
            median_distance_normalized = np.quantile(min_distances_normalized, 0.5)

            # create a DataFrame with min distances
            df_min_distances = pd.DataFrame(min_distances, columns=[os.path.basename(path)])
            df_red_to_nuclei_distances_raw = pd.concat([df_red_to_nuclei_distances_raw, df_min_distances], axis=1) # concatenate along the column
            
            df_min_distances_normalized = pd.DataFrame(min_distances_normalized , columns=[os.path.basename(path)])
            df_red_to_nuclei_distances_normalized = pd.concat([df_red_to_nuclei_distances_normalized, df_min_distances_normalized], axis=1) # concatenate along the column

        # Simulate random distribution of red centroids within the cytosol
        cytosol_coords = np.argwhere(cytosol == True)
        num_red_centroids = len(red_centroids)
        random_indices = np.random.choice(range(len(cytosol_coords)), num_red_centroids)
        random_cytosol_coords = cytosol_coords[random_indices]
        random_cytosol_coords_physical = random_cytosol_coords * voxel_dims
       
        # Calculate distances from random centroids to the nearest nuclei pixel
        if random_cytosol_coords_physical.size and len(coords_Nuclei_physical) > 0:
            nbrs_random = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(coords_Nuclei_physical)
            distances_random, indices_random = nbrs_random.kneighbors(random_cytosol_coords_physical)
            
            min_distances_random = distances_random.flatten()
            min_distances_random_normalized = min_distances_random / max_distance_from_nuclei
            min_distances_random_raw = min_distances_random
            
            # Calculate the median for min_distances_random_normalized for the current path
            median_distance_random_normalized = np.quantile(min_distances_random_normalized, 0.5)
            median_distances_random_raw = np.quantile(min_distances_random_raw, 0.5)
            
            # create a DataFrame with normalized random min distances
            df_min_distances_random_normalized = pd.DataFrame(min_distances_random_normalized, columns=[os.path.basename(path)])
            df_random_red_to_nuclei_distances_normalized = pd.concat([df_random_red_to_nuclei_distances_normalized, df_min_distances_random_normalized], axis=1)  # concatenate along the column
            df_min_distances_random_raw = pd.DataFrame(min_distances_random_raw, columns=[os.path.basename(path)])
            df_random_red_to_nuclei_distances_raw = pd.concat([df_random_red_to_nuclei_distances_raw, df_min_distances_random_raw], axis=1)  # concatenate along the column

#             df_temp['Half_Dist_random_{WORD1}_distance_to_nuclei_{WORD2}_normalized'] = median_distance_random_normalized
            
        # Calculate and add the volume of the Body mask to df1
        puncta_red_total = len(red_centroids)
        
        voxel_volume = x_dim * y_dim * z_dim  # Calculate the volume of a single voxel
        cytosol_volume = np.sum(cytosol) * voxel_volume
        nuclear_volume = np.sum(Nuclei_bool) * voxel_volume
        body_volume = np.sum(Body_bool) * voxel_volume
        nuclear_mRNA_volume = np.sum(mRNA1_bool * Nuclei_bool) * voxel_volume
        cytosol_mRNA_volume = np.sum(mRNA1_bool * cytosol) * voxel_volume
        
        df_temp = pd.DataFrame({
            f'Puncta_absolute_{WORD1}_{WORD2}': [puncta_red_total],
            f'Puncta_Normalized_{WORD1}_{WORD2}': [puncta_red_total / cytosol_volume],
            f'Nuclear_mRNA_Volume_Normalized_{WORD1}_{WORD2}': [nuclear_mRNA_volume / nuclear_volume],
            f'Cytosol_mRNA_Volume_Normalized_{WORD1}_{WORD2}': [cytosol_mRNA_volume / cytosol_volume],
            f'Cytosol_Volume_{WORD1}_{WORD2}': [cytosol_volume],
            
            f'Half_Dist_membrane_distance_to_nuclei_{WORD2}_raw': [median_distance_membrane_raw],
            f'Half_Dist_membrane_distance_to_nuclei_{WORD2}_normalized': [median_distance_membrane_normalized],

            f'Half_Dist_{WORD1}_distance_to_Nuclei_{WORD2}_raw': [median_distance_raw],
            f'Half_Dist_{WORD1}_distance_to_Nuclei_{WORD2}_normalized': [median_distance_normalized],
            
            f'Half_Dist_random_{WORD1}_distance_to_nuclei_{WORD2}_raw': [median_distances_random_raw],            
            f'Half_Dist_random_{WORD1}_distance_to_nuclei_{WORD2}_normalized': [median_distance_random_normalized],

            "path": path
        })

        df = pd.concat([df, df_temp], ignore_index=True)
        
        end_time = time.time() # record end time
        elapsed_time = end_time - start_time # calculate elapsed timeCalm1

        print(f"Processed {file} in {elapsed_time:.2f} seconds")
    
    df.to_csv(f'Puncta_Processing_{WORD1}_to_Nuclei_{WORD2}.csv')       

    df_red_to_nuclei_distances_raw.to_csv(fr'{WORD1}_to_Nuclei_Puncta_raw_distances_{WORD2}.csv') 
    df_random_red_to_nuclei_distances_raw.to_csv(fr'random_{WORD1}_to_Nuclei_Puncta_raw_distances_{WORD2}.csv')     
    df_membrane_to_nuclei_distances_raw.to_csv(fr'membrane_{WORD1}_to_Nuclei_Voxel_raw_distances_{WORD2}.csv')     
    
    df_red_to_nuclei_distances_normalized.to_csv(fr'{WORD1}_to_Nuclei_Puncta_norm_distances_{WORD2}.csv')     
    df_random_red_to_nuclei_distances_normalized.to_csv(fr'random_{WORD1}_to_Nuclei_Puncta_norm_distances_{WORD2}.csv')     
    df_membrane_to_nuclei_distances_normalized.to_csv(fr'membrane_{WORD1}_to_Nuclei_Voxel_norm_distances_{WORD2}.csv')    
    
    return df, df_red_to_nuclei_distances_raw, df_red_to_nuclei_distances_normalized, df_random_red_to_nuclei_distances_normalized

In [1]:
labeled_puncta_path_list = get_labeled_puncta_path("Ryr2", "WT")
print(len(labeled_puncta_path_list))
Puncta_Ryr2_Fig3_WT, Dist_Ryr2_Fig3_WT, Dist_Norm_Ryr2_Fig3_WT, Dist_Random_Norm_Ryr2_Fig3_WT = Puncta_Processing(labeled_puncta_path_list, "Ryr2", "WT")
